## EX4

In [58]:
import pandas as pd
import numpy as np


### Định nghĩa các hàm

In [60]:
def create_train_data():
    data = [
        ['Sunny', 'Hot', 'High', 'Weak', 'no'],
        ['Sunny', 'Hot', 'High', 'Strong', 'no'],
        ['Overcast', 'Hot', 'High', 'Weak', 'yes'],
        ['Rain', 'Mild', 'High', 'Weak', 'yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'no'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'yes'],
        ['Overcast', 'Mild', 'High', 'Weak', 'no'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'yes'],
        ['Rain', 'Mild', 'Normal', 'Weak', 'yes']
    ]
    return np.array(data)


In [71]:
def compute_prior_probability(train_data):
    y_unique = ['no','yes']
    prior_probability = np.zeros(len( y_unique))
    count_no = np.count_nonzero(train_data[:,4] == y_unique[0])
    count_yes = np.count_nonzero(train_data[:,4] == y_unique[1])
    prior_probability[0] = count_no/(count_no + count_yes)
    prior_probability[1] = count_yes/(count_no + count_yes)
    return prior_probability

In [105]:
def compute_conditional_probability(train_data):
    y_unique = ['no','yes']
    x_name_list = []
    conditional_probability_no = []     # list chưa SX tương ứng x_name_list , với điều kiện no
    conditional_probability_yes = []    # list chưa SX tương ứng x_name_list , với điều kiện yes
    # lấy dữ liệu lần lượt tất cả các cột, trừ cột cuối cùng (cột xem là result)
    for i in range (0, train_data.shape[1]-1):
        x_unique = np.unique(train_data[:,i])
        x_name_list.append(x_unique)

        conditional_probability_no.append([0]*len(x_unique))
        conditional_probability_yes.append([0]*len(x_unique))

        for j in range (0, len(x_unique)):
                ## tính toán conditional_probability cho 'no'
            index_no = np.nonzero(train_data[:,train_data.shape[1]-1] == y_unique[0])[0]   # lấy index no cột cuối cùng
            count_j = np.count_nonzero(train_data[:,i][index_no] == x_unique[j])
            conditional_probability_no[i][j] = count_j/len(index_no)

                ## tính toán conditional_probability cho 'yes'
            index_yes = np.nonzero(train_data[:,train_data.shape[1]-1] == y_unique[1])[0]   # lấy index yes cột cuối cùng
            count_j = np.count_nonzero(train_data[:,i][index_yes] == x_unique[j])
            conditional_probability_yes[i][j] = count_j/len(index_yes)

    conditional_probability = [conditional_probability_no,conditional_probability_yes]
    return conditional_probability, x_name_list



In [119]:
def get_index_from_value(value, x_name_list):
    for i in range(len(x_name_list)):
        for j in range (len(x_name_list[i])):
            if x_name_list[i][j] == value:
                return i,j


In [137]:
# ##########################
# Train Naive Bayes Model
# ##########################
def train_naive_bayes(train_data):
    prior_probability = compute_prior_probability(train_data)
    conditional_probability, x_name_list = compute_conditional_probability(train_data)
    return prior_probability, conditional_probability, x_name_list 

### Cách 1: tường minh

In [142]:
# ###################
# Prediction
# ###################
def prediction_play_tennis (given_x, x_name_list , prior_probability , conditional_probability):
    
    x1 = get_index_from_value(given_x[0],x_name_list)
    x2 = get_index_from_value(given_x[1],x_name_list)
    x3 = get_index_from_value(given_x[2],x_name_list)
    x4 = get_index_from_value(given_x[3],x_name_list)

    # gán biến Prediction ban đầu
    p0 = 0  # sx sẽ thực hiện no
    p1 = 0  # sx sẽ thực hiện yes

    # tính sx có điều kiện với 'no':  'p(no).p(x/no)'
    x1_no = conditional_probability[0][x1[0]][x1[1]]
    x2_no = conditional_probability[0][x2[0]][x2[1]]
    x3_no = conditional_probability[0][x3[0]][x3[1]]
    x4_no = conditional_probability[0][x4[0]][x4[1]]
    p_no = prior_probability[0] * x1_no * x2_no * x3_no * x4_no

    # tính sx có điều kiện với 'yes':  'p(yes).p(x/yes)'
    x1_yes = conditional_probability[1][x1[0]][x1[1]]
    x2_yes = conditional_probability[1][x2[0]][x2[1]]
    x3_yes = conditional_probability[1][x3[0]][x3[1]]
    x4_yes = conditional_probability[1][x4[0]][x4[1]]
    p_yes = prior_probability[1] * x1_yes * x2_yes * x3_yes * x4_yes

    # tính không gian mẫu
    p_total = p_no + p_yes

    # tính sx sẽ thực hiên (prediction)
    p0 = p_no/p_total
    p1 = p_yes/p_total

    y_predict = None

    if p0 > p1:
        y_predict = 'NO'
    if p0 < p1:
        y_predict = 'YES'

    return y_predict,p0,p1,p_no,p_yes



### Cách 2: tổng quát hơn

In [153]:
# ###################
# Prediction
# ###################
def prediction_play_tennis_2 (given_x, x_name_list , prior_probability , conditional_probability):
    
    x0 = [get_index_from_value(a,x_name_list) for a in given_x]

    # gán biến Prediction ban đầu
    p0 = 0  # sx sẽ thực hiện no
    p1 = 0  # sx sẽ thực hiện yes

    # tính sx có điều kiện với 'no':  'p(no).p(x/no)'
    x_no = [conditional_probability[0][x[0]][x[1]] for x in x0]
    p_no = prior_probability[0] * np.prod(x_no)
    # np.prod(x_no) nhân các hàng của một vector

    # tính sx có điều kiện với 'yes':  'p(yes).p(x/yes)'
    x_yes = [conditional_probability[1][x[0]][x[1]] for x in x0]
    p_yes = prior_probability[1] * np.prod(x_yes)

    # tính không gian mẫu
    p_total = p_no + p_yes

    # tính sx sẽ thực hiên (prediction)
    p0 = p_no/p_total
    p1 = p_yes/p_total

    y_predict = None

    if p0 > p1:
        y_predict = 'NO'
    if p0 < p1:
        y_predict = 'YES'

    return y_predict,p0,p1,p_no,p_yes

### Test kết quả

In [154]:
train_data = create_train_data()
X = ['Sunny','Cool', 'High', 'Strong']
prior_probability, conditional_probability, x_name_list = train_naive_bayes(train_data)
y_predict,p0,p1,p_no,p_yes = prediction_play_tennis(X, x_name_list , prior_probability , conditional_probability)
print(y_predict,p0,p1,p_no,p_yes)


NO 0.8709677419354839 0.1290322580645161 0.018750000000000003 0.0027777777777777775


In [155]:
train_data = create_train_data()
X = ['Sunny','Cool', 'High', 'Strong']
prior_probability, conditional_probability, x_name_list = train_naive_bayes(train_data)
y_predict,p0,p1,p_no,p_yes = prediction_play_tennis_2 (X, x_name_list , prior_probability , conditional_probability)
print(y_predict,p0,p1,p_no,p_yes)

NO 0.8709677419354839 0.1290322580645161 0.018750000000000003 0.0027777777777777775


In [318]:
train_data = create_train_data()
prior_probability = compute_prior_probability(train_data)
print('sắc xuất play tenis = "no": ',prior_probability[0])
print('sắc xuất play tenis = "yes": ',prior_probability[1])


sắc xuất play tenis = "no":  ['no' 'yes']
sắc xuất play tenis = "yes":  [0.4 0.6]


## EX5:

In [279]:
import numpy as np
import pandas as pd
import math


### Create Functions

In [280]:
def create_data_iris(path):
    data_fr = pd.read_csv(path, header=None)
    header = ['Sepal length','Sepal width','Petal length', 'Petal width', 'Species']
    data_fr.columns=header
    train_data = data_fr.to_numpy()
    return train_data


In [281]:
def compute_prior_probability(train_data):
    last_col = len(train_data[1,:])-1
    y_unique = np.unique(train_data[:,last_col]) #last column in table
    count = [np.count_nonzero(train_data[:,last_col] == y) for y in y_unique ]
    prior_probability = count/np.sum(count)
    return y_unique, prior_probability

In [305]:
def gaussian_function(x,mean, sqrt_var):
    gaus = []
    for i in range(len(mean)):
        p1 = 1/(sqrt_var[i]*np.sqrt(2*np.pi))
        p2 = pow((x[i]-mean[i])/sqrt_var[i],2)
        p3 = np.exp(-p2/2)
        gaus.append(p1*p3) 
    return np.array(gaus)

In [ ]:
def compute_conditional_probability(x, train_data):
    last_col = len(train_data[1,:])-1
    y_unique = np.unique(train_data[:,last_col])
    conditional_probability = []
    for i in range(len(y_unique)):
        index = np.nonzero(train_data[:,train_data.shape[1]-1] == y_unique[i])[0]   # lấy index y_unique[i] cột cuối cùng
        table_i = train_data[index,:train_data.shape[1]-1]
        mean_i = np.mean(table_i, axis=0)
        var_i = np.var(table_i, axis=0)
        sqrt_var_i = []
        for data in var_i:
            sqrt_var_i.append(math.sqrt(data))
        sqrt_var_i = np.array(sqrt_var_i)

        # tính sx của từng hàng
        p_i = gaussian_function(x,mean_i,sqrt_var_i)

        # sx có điều kiện của tưng thành phần
        conditional_probability.append(np.prod(p_i))

    
    return np.array(conditional_probability)

In [320]:
def train_gaussian_naive(x, train_data):
    y_unique, prior_probability = compute_prior_probability(train_data)
    conditional_probability = compute_conditional_probability(x, train_data)
    return y_unique, prior_probability,conditional_probability
    

In [321]:
def predict_data_iris(y_unique,prior_probability,conditional_probability):
    p = prior_probability*conditional_probability
    predict_list = (p/np.sum(p))
    index_predict = np.nonzero(predict_list == np.max(predict_list))[0]
    predict = y_unique[index_predict]
    return predict

### Test result

In [310]:
train_data = create_data_iris('iris.data.csv')
y_unique, prior_probability = compute_prior_probability(train_data)
index = np.nonzero(train_data[:,train_data.shape[1]-1] == y_unique[0])[0]
table_i = train_data[index,:train_data.shape[1]-1]
mean_i = np.mean(table_i, axis=0)
var_i = np.var(table_i, axis=0)
sqrt_var_i = []
for data in var_i:
    sqrt_var_i.append(math.sqrt(data))
sqrt_var_i = np.array(sqrt_var_i)

print(y_unique)
print(y_unique[0])
print(index)
print()
print(mean_i)
print(var_i)
print()
print(type(sqrt_var_i))
#print(np.sqrt(var_i))
print()
print(prior_probability)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
Iris-setosa
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]

[5.005999999999999 3.4180000000000006 1.464 0.2439999999999999]
[0.12176399999999993 0.142276 0.02950400000000001 0.011263999999999996]

<class 'numpy.ndarray'>

[0.33333333 0.33333333 0.33333333]


In [322]:
train_data = create_data_iris('iris.data.csv')
X = [6.3 , 3.3 , 6.0 , 2.5]
y_unique, prior_probability = compute_prior_probability(train_data)
conditional_probability = compute_conditional_probability(X, train_data)
predict  = predict_data_iris(y_unique,prior_probability,conditional_probability)
predict

array(['Iris-virginica'], dtype=object)

In [315]:
X = [5.0 ,2.0 ,3.5 ,1.0]
y_unique, prior_probability = compute_prior_probability(train_data)
conditional_probability = compute_conditional_probability(X, train_data)
predict = predict_data_iris(y_unique,prior_probability,conditional_probability)
predict

array(['Iris-versicolor'], dtype=object)

In [316]:
X = [4.9 ,3.1 ,1.5 ,0.1]
y_unique, prior_probability = compute_prior_probability(train_data)
conditional_probability = compute_conditional_probability(X, train_data)
predict = predict_data_iris(y_unique,prior_probability,conditional_probability)
predict

array(['Iris-setosa'], dtype=object)